In [1]:
import logging
import pandas as pd
import pandas_gbq
from datetime import datetime
from web3 import Web3
from google.oauth2 import service_account

from uniswap_utils import getPoolPrice #https://github.com/amantay-a/misc/blob/master/uniswap_utils.py

token_dict = {'ETH':'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', #WETH
              'BTC':'0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', #WBTC
              'USDC':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
               'DAI':'0x6B175474E89094C44Da98b954EedeAC495271d0F',
             '1INCH':'0x111111111117dC0aa78b770fA6A738034120C302',
              'AAVE':'0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9',
              'COMP':'0xc00e94Cb662C3520282E6f5717214004A7f26888',
               'DPI':'0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b',
               'FEI':'0x956F47F50A910163D8BF957Cf5846D573E7f87CA',
              'LINK':'0x514910771AF9Ca656af840dff83E8264EcF986CA',
               'SNX':'0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F',
               'UNI':'0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
               'YFI':'0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e',
              'LUNA':'0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9', #WLUNA
               'FTM':'0x4E15361FD6b4BB609Fa63C81A2be19d873717870',
             'SUSHI':'0x6B3595068778DD592e39A122f4f5a5cF09C90fE2',
               'CRV':'0xD533a949740bb3306d119CC777fa900bA034cd52',
               'CVX':'0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B',
               'LDO':'0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32',
             'STETH':'0xDFe66B14D37C77F4E9b180cEb433d1b164f0281D',
              'FRAX':'0x853d955aCEf822Db058eb8505911ED77F175b99e',
              'LUSD':'0x5f98805A4E8be255a32880FDeC7F6728C6568bA0',
              'SUSD':'0x57Ab1ec28D129707052df4dF418D58a2D46d5f51',
              'GUSD':'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd',
              'USDT':'0xdAC17F958D2ee523a2206206994597C13D831ec7'
            }
# To be able to call historical prices, RPC must support archive mode, suggest using alchemy as it provide archive rpc for free
RPCEndpoint = 'https://eth-mainnet.alchemyapi.io/v2/FKfen82HuKwIHeV8wXZ5nPFCQ5r_VEt-'

w3 = Web3(Web3.HTTPProvider(RPCEndpoint, request_kwargs={'timeout': 20}))
print(f'Blockchain connected: {w3.isConnected()}')

Blockchain connected: True


In [2]:
def get_uni_price(w3, tokenA, tokenB, block, TWAPWindows, PoolFees):
    
    price_twap = getPoolPrice(w3, 
                              tokenA, 
                              tokenB,
                              TWAPWindows = TWAPWindows,
                              blocks = [block],
                              PoolFees = [3000, 10000] 
                             )
    
    #print(price_twap)
    ret = {}
    for w in TWAPWindows:
        s = 0
        cnt = 0
        for i, f in enumerate(PoolFees):
            price = price_twap[block][i]['twap'][w]
            #print('price', price)
            if price:
                s+= price
                cnt+=1
            #print('s=','s')    
        if cnt>0:        
            ret.update({'twap_'+str(w):s/cnt}) 
        else:
            ret.update({'twap_'+str(w):None})
    return ret

In [3]:
credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'

bq_select = '''with u as (select ticker, max(blockNumber) as twap_max_block from gearbox.uniswap_prices group by ticker) 
               select 
                   cl.ticker, cl.blockNumber, cl.updated_at, cl.price_decimal as price_chainlink
               from 
                   gearbox.oracle_price_history cl
                   left join u on u.ticker = cl.ticker
               where cl.base !='usd'
                   and cl.blockNumber > coalesce(twap_max_block,0)
               order by cl.ticker, cl.blockNumber

            '''
df_cl = pandas_gbq.read_gbq(bq_select, 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
display(df_cl)


,ticker,blockNumber,updated_at,price_chainlink
0,1inch-eth,12098765.0,2021-03-24 01:57:00+00:00,0.002473
1,1inch-eth,12103525.0,2021-03-24 19:39:52+00:00,0.002423
2,1inch-eth,12104265.0,2021-03-24 22:23:09+00:00,0.002374
3,1inch-eth,12105283.0,2021-03-25 02:03:12+00:00,0.002318
4,1inch-eth,12106336.0,2021-03-25 05:55:06+00:00,0.002365
...,...,...,...,...
57994,yfi-eth,14653153.0,2022-04-25 10:02:17+00:00,6.313382
57995,yfi-eth,14653743.0,2022-04-25 12:14:55+00:00,6.376828
57996,yfi-eth,14654593.0,2022-04-25 15:33:56+00:00,6.311908
57997,yfi-eth,14655503.0,2022-04-25 18:55:17+00:00,6.248338


In [4]:
df = df_cl
df['tokenA'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[0].upper()])
df['tokenB'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[1].upper()])
df

,ticker,blockNumber,updated_at,price_chainlink,tokenA,tokenB
0,1inch-eth,12098765.0,2021-03-24 01:57:00+00:00,0.002473,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
1,1inch-eth,12103525.0,2021-03-24 19:39:52+00:00,0.002423,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
2,1inch-eth,12104265.0,2021-03-24 22:23:09+00:00,0.002374,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
3,1inch-eth,12105283.0,2021-03-25 02:03:12+00:00,0.002318,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
4,1inch-eth,12106336.0,2021-03-25 05:55:06+00:00,0.002365,0x111111111117dC0aa78b770fA6A738034120C302,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...,...,...
57994,yfi-eth,14653153.0,2022-04-25 10:02:17+00:00,6.313382,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
57995,yfi-eth,14653743.0,2022-04-25 12:14:55+00:00,6.376828,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
57996,yfi-eth,14654593.0,2022-04-25 15:33:56+00:00,6.311908,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
57997,yfi-eth,14655503.0,2022-04-25 18:55:17+00:00,6.248338,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


In [ ]:
twapwindows =  [1, 60*15, 60*20]
twapwindow_cols = ['twap_'+str(x) for x in twapwindows]
poolfee = [3000, 5000]
df[twapwindow_cols] = None
cnt = 0
print(datetime.utcnow(),'Start')
for ticker in df['ticker'].unique():
    df_ticker = df[df['ticker']==ticker].copy()
    print(datetime.utcnow(),f'{cnt}/{len(df)} ({ticker})..')
    for x in df_ticker.itertuples():
        prices = get_uni_price(w3, x.tokenA, x.tokenB, int(x.blockNumber), twapwindows, poolfee)
        prices = list(prices.values())
        df_ticker.loc[(df['blockNumber'] == x.blockNumber)&(df['ticker'] == x.ticker), twapwindow_cols] = prices
        cnt+=1
        if cnt%100==0:
            print(datetime.utcnow(),f'{cnt}/{len(df)} ({ticker})..')
    df_ticker[twapwindow_cols] = df_ticker[twapwindow_cols].astype('float64')
    pandas_gbq.to_gbq(df_ticker, 
                      'gearbox.uniswap_prices',
                      project_id=gcp_project_id,
                      if_exists = 'append', 
                      progress_bar = False)
display(df)

2022-04-26 08:26:13.309642 Start
2022-04-26 08:28:03.037126 100/57999..
2022-04-26 08:29:49.787678 200/57999..
2022-04-26 08:31:26.283569 300/57999..
2022-04-26 08:32:25.168829 400/57999..
2022-04-26 08:33:22.776177 500/57999..
2022-04-26 08:34:28.210969 600/57999..
2022-04-26 08:35:33.867790 700/57999..
2022-04-26 08:36:38.143924 800/57999..
2022-04-26 08:37:43.492524 900/57999..
2022-04-26 08:38:47.890821 1000/57999..
2022-04-26 08:39:52.491822 1100/57999..
2022-04-26 08:40:56.100295 1200/57999..
2022-04-26 08:42:01.829987 1300/57999..
2022-04-26 08:43:06.442476 1400/57999..
2022-04-26 08:44:10.966009 1500/57999..
2022-04-26 08:45:11.377041 1600/57999..
2022-04-26 08:46:12.250701 1700/57999..
2022-04-26 08:48:00.275120 1800/57999..
2022-04-26 08:49:49.390899 1900/57999..
2022-04-26 08:51:20.021684 2000/57999..
2022-04-26 08:52:47.125521 2100/57999..
2022-04-26 08:54:02.737545 2200/57999..
2022-04-26 08:55:08.872292 2300/57999..
2022-04-26 08:56:20.931043 2400/57999..
2022-04-26 08:57

In [ ]:
df